In [15]:
import woslite_client
import json

In [16]:
from __future__ import print_function
import time
import woslite_client
from woslite_client.rest import ApiException
from pprint import pprint


# Configure API key authorization: key
configuration = woslite_client.Configuration()
configuration.api_key['X-ApiKey'] = '7263295125113a07f088770334f9f70748c1ac2a'

In [22]:
#create an instance of the API class
# Springer Nature
# Royal Soc Chemistry
# Elsevier
# Article OR Review Article OR Proceeding Paper
integration_api_instance = woslite_client.IntegrationApi(woslite_client.ApiClient(configuration))
search_api_instance = woslite_client.SearchApi(woslite_client.ApiClient(configuration))
database_id = 'WOS'  # str | Database to search. Must be a valid database ID, one of the following: BCI/BIOABS/BIOSIS/CCC/DCI/DIIDW/MEDLINE/WOK/WOS/ZOOREC. WOK represents all databases.
usr_query = 'ALL=(copper paste OR Cu paste) AND PUBL=(Elsevier)'  # str | User query for requesting data, ex: TS=(cadmium). The query parser will return errors for invalid queries.
count = 100 # int | Number of records returned in the request
first_record = 1  # int | Specific record, if any within the result set to return. Cannot be less than 1 and greater than 100000.
publish_time_span = '2000-01-01+2023-09-01'
lang = 'en'  # str | Language of search. This element can take only one value: en for English. If no language is specified, English is passed by default. (optional)
sort_field = 'PY+D'  # str | Order by field(s). Field name and order by clause separated by '+', use A for ASC and D for DESC, ex: PY+D. Multiple values are separated by comma. (optional)

## only get doi

In [23]:
def get_doi(response):
    dl = []
    for i in response.data:
        doi = i.other.identifier_doi
        x_doi = i.other.identifier_xref_doi
        if doi != None:
            dl.append(doi[0])
        else:
            if x_doi != None:
                dl.append(x_doi[0])       
    return dl

In [24]:
total_doi = []

In [25]:
from tqdm import tqdm

# 100000
try:
    api_response = search_api_instance.root_get(database_id, usr_query, count, first_record, lang=lang,
                                                         sort_field=sort_field, publish_time_span=publish_time_span)
    query_id = api_response.query_result.query_id
    print(api_response.query_result.records_found)
    if api_response.query_result.records_found < 100000:        
        doi_100 = get_doi(api_response)
        total_doi.extend(doi_100)
        for i in tqdm(range(101, api_response.query_result.records_found, 100)):
            api_response = search_api_instance.query_query_id_get(query_id, 100, i)
            doi_post = get_doi(api_response)
            total_doi.extend(doi_post)
    else:
        print("too large")

except ApiException as e:
    print("Exception when calling SearchApi->root_get: {}".format(e))

  0%|          | 0/12 [00:00<?, ?it/s]

1217


100%|██████████| 12/12 [00:05<00:00,  2.21it/s]


In [13]:
print(len(total_doi))
tmp = list(set(total_doi))
print(len(total_doi))

367
367


In [26]:
json_str = json.dumps(total_doi, indent=4)
with open('cp_elsevier_1217.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

In [59]:
import os
from tqdm import tqdm

ts_doi = []
ts_em = os.listdir('D:/个人/scrawler/Energy_bert_v2/ts(energy&material)')
for te in tqdm(ts_em):
    if 'Elsevier' in te:
        file = 'D:/个人/scrawler/Energy_bert_v2/ts(energy&material)/'+te
        with open(file, 'r', encoding='utf-8') as f:
            doi_list = json.load(f)
            ts_doi.extend(doi_list)
allnotts_doi = []
all_em = os.listdir('D:/个人/scrawler/Energy_bert_v2/all(energy&material)')
for a in all_em:
    file = 'D:/个人/scrawler/Energy_bert_v2/all(energy&material)/'+a
    with open(file, 'r', encoding='utf-8') as f:
        doi_list = json.load(f)
        for d in tqdm(doi_list):
            if d not in ts_doi:
                allnotts_doi.append(d)
print(len(allnotts_doi))

100%|██████████| 19851/19851 [00:46<00:00, 431.46it/s]

236619


In [60]:
json_str = json.dumps(allnotts_doi, indent=4)
with open('D:/个人/scrawler/Energy_bert_v2/ALLNOTTS_Elsevier_236619.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

## get doi with year

In [4]:
def get_doi_year(response):
    dl = {}
    for i in response.data:
        doi = i.other.identifier_doi
        x_doi = i.other.identifier_xref_doi
        year = i.source.published_biblio_year
        if year != None:
            if doi != None:
                dl[doi[0]]=year[0]
            else:
                if x_doi != None:
                    dl[x_doi[0]]=year[0]           
    return dl

In [30]:
total_doi_year = {}

In [31]:
from tqdm import tqdm


try:
    api_response = search_api_instance.root_get(database_id, usr_query, count, first_record, lang=lang,
                                                         sort_field=sort_field, publish_time_span=publish_time_span)
    query_id = api_response.query_result.query_id
    print(api_response.query_result.records_found)
    doi_100 = get_doi(api_response)
    total_doi_year.update(doi_100)
    for i in tqdm(range(101, api_response.query_result.records_found, 100)):
        api_response = search_api_instance.query_query_id_get(query_id, 100, i)
        doi_post = get_doi(api_response)
        total_doi_year.update(doi_post)

except ApiException as e:
    print("Exception when calling SearchApi->root_get: {}".format(e))

  0%|          | 0/973 [00:00<?, ?it/s]

97303


100%|██████████| 973/973 [06:05<00:00,  2.66it/s]


In [32]:
print(len(total_doi_year))

77363


In [18]:
json_str = json.dumps(total_doi_year, indent=4)
with open('total_doi_year.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

## get info of doi

In [12]:
from __future__ import print_function
import time
import woslite_client
from woslite_client.rest import ApiException
from pprint import pprint


def doi_keywords(doi):
    # Configure API key authorization: key
    configuration = woslite_client.Configuration()
    configuration.api_key['X-ApiKey'] = '7263295125113a07f088770334f9f70748c1ac2a'
    api_instance = woslite_client.SearchApi(woslite_client.ApiClient(configuration))
    database_id = 'WOS' # str | Database to search. Must be a valid database ID, one of the following: BCI/BIOABS/BIOSIS/CCC/DCI/DIIDW/MEDLINE/WOK/WOS/ZOOREC. WOK represents all databases.
    usr_query = 'DO=('+doi+')'# str | User query for requesting data, ex: TS=(cadmium). The query parser will return errors for invalid queries.
    count = 1 # int | Number of records to return, must be 0-100.
    first_record = 1 # int | Specific record, if any within the result set to return. Cannot be less than 1 and greater than 100000.
    
    try:
        api_response = api_instance.root_get(database_id, usr_query,count, first_record)
        # print(api_response.data[0].doctype.doctype[0])
        # return api_response.data[0].source.source_title[0]
        # print(api_response.data[0])
        return api_response.data[0].keyword.keywords
    except:
        return 'y'   

In [9]:
doi_keywords('10.1016/j.aca.2012.12.019')

['Natural deep eutectic solvents',
 'Ionic liquids',
 'Physicochemical properties',
 'Green technology',
 'Solubility']

In [13]:
import json
from tqdm import tqdm

doi_key = {}
with open('1000.json', 'r', encoding='utf-8') as f:
    dois = json.load(f)
for d in tqdm(dois):
    doi_key[d] = doi_keywords(d)

100%|██████████| 993/993 [17:55<00:00,  1.08s/it]


In [14]:
json_str = json.dumps(doi_key, indent=4)
with open('doi_key.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)